# Application of Bootstrap samples in Random Forest

In [1]:
import numpy as np
import pandas as pd
import random 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
boston = fetch_openml(name='boston', version=1, as_frame=True)
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

C:\Users\svani\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
x=boston.data #independent variables
y=boston.target #target variable

In [3]:
x.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33


In [4]:
y.head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: MEDV, dtype: float64

In [5]:
x.shape, y.shape

((506, 13), (506,))

## <font color='red'><b>Task 1</b></font>

<font color='red'> <b>Step - 1</b></font>

*  <font color='blue'><b>Creating samples</b></font><br>
    <b> Randomly create 30 samples from the whole boston data points</b>
    *  Creating each sample: Consider any random 303(60% of 506) data points from whole data set and then replicate any 203 points from the sampled points
    
     For better understanding of this procedure lets check this examples, assume we have 10 data points [1,2,3,4,5,6,7,8,9,10], first we take 6 data points randomly , consider we have selected [4, 5, 7, 8, 9, 3] now we will replicate 4 points from [4, 5, 7, 8, 9, 3], consder they are [5, 8, 3,7] so our final sample will be [4, 5, 7, 8, 9, 3, 5, 8, 3,7]
* <font color='blue'><b> Create 30 samples </b></font>
    *  Note that as a part of the Bagging when you are taking the random samples <b>make sure each of the sample will have different set of columns</b><br>
Ex: Assume we have 10 columns[1 ,2 ,3 ,4 ,5 ,6 ,7 ,8 ,9 ,10] for the first sample we will select [3, 4, 5, 9, 1, 2] and for the second sample  [7, 9, 1, 4, 5, 6, 2] and so on...
Make sure each sample will have atleast 3 feautres/columns/attributes

* <font color='red'><b> Note - While selecting the random 60% datapoints from the whole data, make sure that the selected datapoints are all exclusive, repetition is not allowed. </b></font>

In [6]:
def generating_samples(input_data, target_data): 
    
    np.random.seed(123) # Set random seed

    # random.choice to generate random indices without replacement
    # Selecting_rows => Getting 303 random row indices from the input_data, without replacement
    Selecting_rows = input_data.sample(n=303, replace=False).index

    # now we will replicate 203 points from above selected rows
    # Replacing Rows => Extracting 206 random row indices from the Selecting_rows
    Replacing_rows = pd.Series(Selecting_rows).sample(n=203, replace=False).values

    # Selecting_columns => Getting 3 to 13 random column indices from input_data
    Selecting_columns = random.randint(3, 13)
    columns_selected = np.array(random.sample(range(0, 13), Selecting_columns ))

    # sample_data => input_data.iloc[Selecting_rows[:,None], Selecting_columns]
    sample_data = input_data.iloc[Selecting_rows, columns_selected]
    
    # target_of_sample_data => target_data[Selecting_rows]
    target_of_sample_data = target_data[Selecting_rows]
    
    # Replicating data
    # Now Replication of Data for 203 data points out of 303 selected points
    replicated_sample_data = input_data.iloc[Replacing_rows, columns_selected ]
    target_of_replicated_sample_data = target_data[Replacing_rows]
    
    # Concatenating data
    final_sample_data = np.vstack((sample_data.values, replicated_sample_data.values ))
    final_target_data = np.vstack((target_of_sample_data.to_numpy().reshape(-1, 1), target_of_replicated_sample_data.to_numpy().reshape(-1, 1)))
            
    return final_sample_data, final_target_data, Selecting_rows, columns_selected

a,b,c,d = generating_samples(x, y)

In [7]:
print(a.shape, b.shape, c.shape, d.shape)

(506, 11) (506, 1) (303,) (11,)


<font color='cyan'> <b> Grader function - 1 </b> </fongt>

In [8]:
def grader_samples(a,b,c,d):
    length = (len(a)==506  and len(b)==506)
    sampled = (len(a)-len(set([str(i) for i in a]))==203)
    rows_length = (len(c)==303)
    column_length= (len(d)>=3)
    assert(length and sampled and rows_length and column_length)
    return True
grader_samples(a,b,c,d)

True

*  <font color='blue'> <b>Create 30 samples </b>

In [9]:
# Create empty lists to store the samples
list_input_data = []
list_output_data = []
list_selected_row = []
list_selected_columns = []

# Loop to generate 30 samples
for i in range(30):
    # Generate a sample
    a, b, c, d = generating_samples(x, y)
    
    # Append the sample to the respective lists
    list_input_data.append(a)
    list_output_data.append(b)
    list_selected_row.append(c)
    list_selected_columns.append(d)

<font color='cyan'> <b>Grader function - 2 </b></font>

In [10]:
def grader_30(a):
    assert(len(a)==30 and len(a[0])==506)
    return True
grader_30(list_input_data)

True

<font color='red'><b>Step - 2 </b></font>

<font color='blue'><b>Building High Variance Models on each of the sample and finding train MSE value</b></font>

*  Build a regression trees on each of 30 samples.
*  Computed the predicted values of each data point(506 data points) in your corpus.
*  Predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{30}\sum_{k=1}^{30}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$
*  Now calculate the $MSE =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$

In [11]:
# Create an empty list to store the trained models
tree_models = []

# Train a decision tree model for each of the 30 samples
for i in range(30):
    tree_model = DecisionTreeRegressor()
    tree_model.fit(list_input_data[i], list_output_data[i])
    
    # Append the trained model to the list
    tree_models.append(tree_model)

In [12]:
print(tree_models)

[DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor(), DecisionTreeRegressor()]


In [13]:
# Make predictions for each model and store the predicted values in a list
list_predicted_values = []
for i in range(30):
    # Predict the output values for the i-th sample using the corresponding model
    predicted_values = tree_models[i].predict(list_input_data[i])
    list_predicted_values.append(predicted_values)  

After getting predicted_y for each data point, we can use sklearns mean_squared_error to calculate the MSE between predicted_y and actual_y.

In [14]:
# Calculate the mean squared error for each sample's predictions and store them in a list

mse_values = []
for i in range(len(list_predicted_values)):
    # Calculate the mean squared error between the i-th sample's actual output and predicted output
    mse = mean_squared_error(list_output_data[i], list_predicted_values[i])
    mse_values.append(mse)
    

print(len(mse_values))
print('-'*50)

print(mse_values)
print('-'*50)


# Calculate the mean of the mean squared errors
mean_mse = np.mean(mse_values)
print('mean of the mean squared errors: ', mean_mse)


30
--------------------------------------------------
[0.0, 0.0, 0.0, 0.0, 7.483265504072839e-32, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06800395256917001, 7.483265504072839e-32, 0.0, 0.0, 0.0, 0.0, 7.483265504072839e-32, 0.0, 7.483265504072839e-32, 0.0, 0.0, 0.0, 0.0, 7.483265504072839e-32, 0.0, 7.483265504072839e-32, 7.483265504072839e-32, 0.0, 0.0]
--------------------------------------------------
mean of the mean squared errors:  0.002266798418972334


It is clearly visible that, the mean squared error (MSE) values for the predicted house prices compared to the actual prices are remarkably low. This indicates that the decision tree models I trained on the Boston Housing dataset are performing exceptionally well in predicting the target variable. The MSE measures the average squared difference between the predicted and actual values, and a lower MSE signifies superior model performance.

Upon examining the MSE values, it is evident that the majority of them are very close to zero. This implies that the models have successfully captured the underlying patterns and relationships within the dataset, enabling them to make highly accurate predictions. The models have effectively learned from the input features and established strong correlations with the target variable.

By calculating the mean of the MSE values, I obtained a value of 0.0002905138339920953. 
This further reinforces the overall outstanding performance of the models. A low mean MSE indicates that, on average, the predicted house prices closely align with the actual prices.


<font color='red'> <b>Step - 3 </b></font>

*  <font color='blue'><b>Calculating the OOB score </b></font>

*  Predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{k}\sum_{\text{k= model which was buit on samples not included } x^{i}}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.
*  Now calculate the $OOB Score =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.

In [15]:
# Create an empty list to store the OOB indices for each tree
oob_indices_per_tree = []

for i in range(30):
    # Get the indices of the rows that were not selected for the i-th sample
    oob_indices = list(set(range(506)) - set(list_selected_row[i]))
    
    # Append the indices to the list
    oob_indices_per_tree.append(oob_indices)

print(len(oob_indices_per_tree))
print(oob_indices_per_tree)

30
[[1, 2, 3, 8, 14, 16, 17, 18, 25, 27, 32, 39, 40, 43, 45, 46, 47, 50, 51, 56, 57, 58, 60, 62, 63, 64, 65, 67, 68, 70, 73, 76, 77, 83, 84, 86, 87, 88, 92, 96, 97, 98, 99, 100, 103, 106, 109, 110, 111, 112, 113, 116, 118, 119, 122, 123, 126, 129, 130, 133, 135, 137, 139, 140, 141, 146, 149, 153, 154, 158, 168, 169, 174, 176, 180, 183, 186, 187, 193, 195, 197, 198, 205, 206, 208, 213, 214, 215, 222, 224, 225, 230, 233, 238, 243, 244, 247, 250, 251, 253, 254, 255, 256, 257, 259, 262, 265, 268, 269, 270, 271, 278, 281, 283, 290, 294, 296, 300, 301, 302, 303, 304, 305, 307, 311, 312, 314, 315, 319, 321, 322, 323, 324, 325, 330, 331, 334, 336, 339, 340, 342, 348, 349, 350, 353, 354, 355, 357, 358, 359, 360, 361, 363, 365, 366, 371, 373, 377, 380, 382, 386, 387, 390, 391, 394, 395, 396, 398, 400, 404, 406, 409, 411, 414, 418, 420, 424, 428, 430, 438, 439, 445, 451, 453, 454, 459, 462, 464, 471, 473, 475, 476, 477, 480, 488, 489, 492, 496, 497, 499, 503, 504, 505], [1, 2, 3, 8, 14, 16, 17, 1

In [16]:
# Create an empty list to store the OOB predictions for each tree
oob_predictions_per_tree = []

for i in range(30):
    # Get the indices of the rows that were not selected for the i-th sample
    oob_indices = oob_indices_per_tree[i]
    
    # Get the input data corresponding to the OOB indices
    oob_input_data = x.iloc[oob_indices, :]
    
    # Select only the columns used for training the i-th model
    oob_input_data = oob_input_data.iloc[:, list_selected_columns[i]]
    
    # Get the predictions on the OOB input data using the i-th model
    oob_predictions = tree_models[i].predict(oob_input_data)

    # Append the predictions to the list
    oob_predictions_per_tree.append(oob_predictions)

print(len(oob_predictions_per_tree))

30


In [17]:
# Create an empty list to store the OOB MSE values for each tree
oob_mse_values = []

# iterate over each tree for calculating the OOB MSE
for i in range(30):
    # Get the OOB indices for the i-th tree
    oob_indices = oob_indices_per_tree[i]
    
    # Get the actual output values corresponding to the OOB indices
    actual_output = y[oob_indices]
    
    # Get the OOB predictions for the i-th tree
    oob_predictions = oob_predictions_per_tree[i]
    
    # Calculate the OOB MSE for the i-th tree
    oob_mse = mean_squared_error(actual_output, oob_predictions)
    
    # Append the OOB MSE to the list
    oob_mse_values.append(oob_mse)
    
print(len(oob_mse_values))
print('-'*50)

print(oob_mse_values)
print('-'*50)

# Calculate the mean of the OOB MSE values
mean_oob_mse = np.mean(oob_mse_values)
print('mean of the OOB MSE values: ', mean_oob_mse)


30
--------------------------------------------------
[52.93014778325123, 23.724876847290638, 31.650492610837436, 17.05128078817734, 26.342315270935963, 15.675418719211823, 73.74600985221674, 19.52857142857143, 49.99733990147784, 19.23295566502463, 62.0695566502463, 30.308091133004925, 40.020640394088666, 42.55935960591133, 30.76443349753694, 16.365320197044333, 29.15630541871921, 19.05192118226601, 71.77669950738915, 18.569901477832513, 19.03408866995074, 25.45729064039409, 21.533743842364533, 32.60310344827587, 19.008768472906404, 19.973152709359603, 43.82600985221675, 19.64576354679803, 40.15049261083744, 78.75039408866995]
--------------------------------------------------
mean of the OOB MSE values:  33.68348152709359


Here, the out-of-bag mean squared error (OOB MSE) values for each tree in the random forest are varied, ranging from 17.247881773399012 to 76.57571428571427.

After calculating the OOB MSE values for all the trees in the random forest, I obtained a mean OOB MSE of 30.2547816091954. This value indicates the overall performance of the random forest model in predicting house prices.

Considering the range of OOB MSE values and the mean value, it can be concluded that the random forest model exhibits some variability in its performance across different trees. While some trees achieve relatively low OOB MSE values, indicating accurate predictions, others have higher OOB MSE values, indicating less accurate predictions.

# <font color='red'><b>Task 2</b></font>

*  <font color='blue'><b>Computing CI of OOB Score and Train MSE</b></font>
  *   Repeat Task 1 for 35 times, and for each iteration store the Train MSE and OOB score </li>
<li> After this we will have 35 Train MSE values and 35 OOB scores </li>
<li> using these 35 values (assume like a sample) find the confidence intravels of MSE and OOB Score </li>
<li> you need to report CI of MSE and CI of OOB Score </li>
<li> Note: Refer the Central_Limit_theorem.ipynb to check how to find the confidence intravel</li>
</ol>

In [18]:
num_samples = 30
num_data_points = 506
num_repetitions = 35

# Create an empty list to store the trained models
tree_models = []

# Train a decision tree model for each of the 30 samples
for i in range(num_samples):
    tree_model = DecisionTreeRegressor()
    tree_model.fit(list_input_data[i], list_output_data[i])
    
    # Append the trained model to the list
    tree_models.append(tree_model)

# Create empty lists to store the MSE scores and OOB scores for each iteration
list_mse_scores = []
list_oob_scores = []

for iteration in range(num_repetitions):
    # Make predictions for each model and store the predicted values in a list
    list_predicted_values = []
    for i in range(num_samples):
        # Predict the output values for the i-th sample using the corresponding model
        predicted_values = tree_models[i].predict(list_input_data[i])
        list_predicted_values.append(predicted_values)

    # Calculate the mean squared error for each sample's predictions and store them in a list
    mse_values = []
    for i in range(len(list_predicted_values)):
        # Calculate the mean squared error between the i-th sample's actual output and predicted output
        mse = mean_squared_error(list_output_data[i], list_predicted_values[i])
        mse_values.append(mse)

    # Append the MSE scores to the list
    list_mse_scores.append(mse_values)

    # Create an empty list to store the OOB indices for each tree
    oob_indices_per_tree = []

    for i in range(num_samples):
        # Get the indices of the rows that were not selected for the i-th sample
        oob_indices = list(set(range(num_data_points)) - set(list_selected_row[i]))

        # Append the indices to the list
        oob_indices_per_tree.append(oob_indices)

    # Create an empty list to store the OOB predictions for each tree
    oob_predictions_per_tree = []

    for i in range(num_samples):
        # Get the indices of the rows that were not selected for the i-th sample
        oob_indices = oob_indices_per_tree[i]

        # Get the input data corresponding to the OOB indices
        oob_input_data = x.iloc[oob_indices, :]

        # Select only the columns used for training the i-th model
        oob_input_data = oob_input_data.iloc[:, list_selected_columns[i]]

        # Get the predictions on the OOB input data using the i-th model
        oob_predictions = tree_models[i].predict(oob_input_data)

        # Append the predictions to the list
        oob_predictions_per_tree.append(oob_predictions)

    # Create an empty list to store the OOB MSE values for each tree
    oob_mse_values = []

    # Iterate over each tree for calculating the OOB MSE
    for i in range(num_samples):
        # Get the OOB indices for the i-th tree
        oob_indices = oob_indices_per_tree[i]

        # Get the actual output values corresponding to the OOB indices
        actual_output = y[oob_indices]

        # Get the OOB predictions for the i-th tree
        oob_predictions = oob_predictions_per_tree[i]

        # Calculate the OOB MSE for the i-th tree
        oob_mse = mean_squared_error(actual_output, oob_predictions)

        # Append the OOB MSE to the list
        oob_mse_values.append(oob_mse)

    # Append the OOB scores to the list
    list_oob_scores.append(oob_mse_values)
    
print(len(list_mse_scores))
print(len(list_oob_scores))


35
35


In [19]:
from scipy.stats import norm

# Calculate the mean and standard deviation of MSE
mse_mean = np.mean(list_mse_scores)
mse_std = np.std(list_mse_scores)

# Calculate the mean and standard deviation of OOB scores
oob_mean = np.mean(list_oob_scores)
oob_std = np.std(list_oob_scores)

# Set the desired confidence level (e.g., 95%)
confidence_level = 0.95

# Calculate the critical value based on the desired confidence level
critical_value = norm.ppf(1 - (1 - confidence_level) / 2)

# Calculate the margin of error for MSE
mse_margin_of_error = critical_value * (mse_std / np.sqrt(num_repetitions))

# Calculate the confidence interval for MSE
mse_ci_lower = mse_mean - mse_margin_of_error
mse_ci_upper = mse_mean + mse_margin_of_error

# Calculate the margin of error for OOB scores
oob_margin_of_error = critical_value * (oob_std / np.sqrt(num_repetitions))

# Calculate the confidence interval for OOB scores
oob_ci_lower = oob_mean - oob_margin_of_error
oob_ci_upper = oob_mean + oob_margin_of_error

# Print the results
print("MSE Mean: {:.4f}".format(mse_mean))
print("MSE Standard Deviation: {:.4f}".format(mse_std))
print("MSE Confidence Interval: [{:.4f}, {:.4f}]".format(mse_ci_lower, mse_ci_upper))
print("---")
print("OOB Mean: {:.4f}".format(oob_mean))
print("OOB Standard Deviation: {:.4f}".format(oob_std))
print("OOB Confidence Interval: [{:.4f}, {:.4f}]".format(oob_ci_lower, oob_ci_upper))

MSE Mean: 0.0023
MSE Standard Deviation: 0.0122
MSE Confidence Interval: [-0.0018, 0.0063]
---
OOB Mean: 33.9146
OOB Standard Deviation: 18.6490
OOB Confidence Interval: [27.7363, 40.0930]


For the mean squared error (MSE), I got a confidence level of 95%, the confidence interval for the MSE is estimated to be [-0.0002, 0.0008]. This interval provides a range within which the true population mean MSE is likely to fall.
Also, the average MSE is 0.0003 with a standard deviation of 0.0016.

While, the out-of-bag (OOB) scores, shows that the average OOB score is 30.0754 with a standard deviation of 16.3830. 
The confidence interval for the OOB scores is [24.6478, 35.5030] at a 95% confidence level.

Comparing the MSE and OOB MSE results, it is evident that the magnitudes of the values differ significantly. 

The MSE measures the performance of the individual decision tree models on their respective training samples, while the OOB MSE evaluates the performance of the random forest model using the out-of-bag samples. 

The MSE values are substantially lower (0.0003) compared to the OOB MSE values (30.0754), indicating that the individual decision tree models tend to overfit the training data. 

This suggests that the random forest model achieves better generalization by considering the OOB samples, resulting in slightly higher MSE values but improved predictive performance overall.

# <font color='red'><b>Task 3</b></font>

*  <font color='blue'><b>Given a single query point predict the price of house.



Consider xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60] 
Predict the house price for this point as mentioned in the step 2 of Task 1.

In [20]:
xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60]

predicted_values = []

# Iterate over each model and make predictions for the data point xq
for i in range(len(tree_models)):
    # Get the selected columns for the i-th model
    selected_columns = list_selected_columns[i]
    
    # Select the corresponding features from xq
    xq_selected = [xq[j] for j in selected_columns]
    
    # Make a prediction using the i-th model
    prediction = tree_models[i].predict([xq_selected])
    
    # Append the prediction to the list of predicted values
    predicted_values.append(prediction)

# Calculate the average predicted value
average_prediction = np.mean(predicted_values)

# Print the predicted house price
print("Predicted House Price:", average_prediction)

Predicted House Price: 18.533333333333335


The prediction is obtained by applying the trained decision tree models to the data point xq. Each decision tree model selects specific features and makes an individual prediction. The predicted values from all the models are then averaged to obtain the final prediction.